In [ ]:
from typing import Literal # Requires Python 3.8+
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib

from parcellation_surface.visualization import visualize_brain_surface
from parcellation_surface.smoothing import smooth_surface_graph
from parcellation_surface.watershed import watershed_by_flooding
from parcellation_surface.gradient import build_mesh_graph
from parcellation_surface.group_analysis import create_random_parcels, homogeneity_craddock_rt


In [ ]:
def extract_fmri_timeseries(dataset = Literal["PPSFACE_N18", "PPSFACE_N20"],
                            hemisphere = Literal["lh", "rh"],
                            run = Literal["1","2"]):
    # Load all of the surf fmri data
    surf_fmri_list = []
    dataset_dir = f"D:\Data_Conn_Preproc\\{dataset}"
    if dataset == "PPSFACE_N18":
        n = 19
    else:
        n = 21
    for i in range(1,n):
        if i == 5 and dataset=="PPSFACE_N20": # Subject 5 is missing
            continue
        subject = f"{i:02d}"
        subj_dir = dataset_dir + r"\sub-" + subject
        fmri_path = subj_dir + f"\\func\surf_conn_sub{subject}_run{run}_{hemisphere}.func.fsaverage6.mgh"
        surf_fmri_img = nib.load(fmri_path)
        surf_fmri = surf_fmri_img.get_fdata()
        surf_fmri = np.squeeze(surf_fmri) # just rearrange the MGH data
        # Normilize the data
        surf_fmri = (surf_fmri - np.mean(surf_fmri, axis=1, keepdims=True)) / np.std(surf_fmri, axis=1, keepdims=True)
        # Replace nan values with 0
        surf_fmri = np.nan_to_num(surf_fmri)
        surf_fmri_list.append(surf_fmri)
    return surf_fmri_list

def extract_parcellation(dataset = Literal["PPSFACE_N18", "PPSFACE_N20"], 
                         hemi = Literal["lh", "rh"], 
                         run = Literal["1","2"]):
    # Load the parcellation
    # bound_path = f"D:\Data_Conn_Preproc\PPSFACE_N20\\boundary_smoothed_map_allsub_run{run}_{hemi}.npy"
    # grad_path = f"D:\Data_Conn_Preproc\PPSFACE_N20\gradients_smoothed_allsub_run{run}_{hemi}.npy"
    bound_path = f"D:\Data_Conn_Preproc\{dataset}\group_parcellation\\boundary_map_group_run{run}_{hemi}.npy"
    grad_path = f"D:\Data_Conn_Preproc\{dataset}\group_parcellation\gradients_smoothed_group_run{run}_{hemi}.npy"
    bound = np.load(bound_path)
    grad = np.load(grad_path)
    return bound, grad

In [ ]:
# Create and save the parcellations
# For ppsface n18
ppsface = "PPSFACE_N20"
for hemi in ["lh", "rh"]:
   surface_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\{hemi}.white"
   inf_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\{hemi}.inflated"
   # surface_path = r"D:\Data_Conn_Preproc\fsaverage6\surf\rh.inflated"
   coords, faces = nib.freesurfer.read_geometry(surface_path)
   coords_, faces_ = nib.freesurfer.read_geometry(inf_path)
   graph = build_mesh_graph(faces)
   bound_sum = 0
   for run in ["1","2"]:
      # Extract the parcellation
      bound_sum += np.load(f"D:\Data_Conn_Preproc\{ppsface}\group_parcellation\\boundary_map_group_run{run}_{hemi}.npy")
        
   # Create the parcels
   bound_smoothed = smooth_surface_graph(graph, bound_sum, iterations=10)
   label_bound = watershed_by_flooding(graph, bound_smoothed)
   # Save the parcels
   np.save(f"D:\Data_Conn_Preproc\{ppsface}\group_parcellation\labels_group_run12_smooth10_{hemi}.npy", label_bound)

In [ ]:
hemi = 'lh'
label = np.load(f"D:\Data_Conn_Preproc\{ppsface}\group_parcellation\labels_group_run12_smooth10_{hemi}.npy")
print((np.unique(label)>=0).sum())

# Show a parcellation

In [ ]:
inf_lh_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\lh.inflated"
pial_lh_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\lh.white"
coords_lh, faces_lh = nib.freesurfer.read_geometry(inf_lh_path)
coords_lh, faces_lh = nib.freesurfer.read_geometry(inf_lh_path)
graph = build_mesh_graph(faces_lh)
label = np.load(f"D:\Data_Conn_Preproc\{ppsface}\group_parcellation\labels_group_run12_smooth10_lh.npy")
# label = create_random_parcels(graph, 240)
visualize_brain_surface(coords_lh, faces_lh, label)

## Show both hemisphere boundary maps

In [ ]:
from visualization import combine_surfaces
# Load the parcellation for lh run1
run = "1"
dataset = "PPSFACE_N18"
white_lh_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\lh.white"
white_rh_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\\rh.white"
# surface_path = r"D:\Data_Conn_Preproc\fsaverage6\surf\rh.inflated"
coords_lh, faces_lh = nib.freesurfer.read_geometry(white_lh_path)
coords_rh, faces_rh = nib.freesurfer.read_geometry(white_rh_path)

# Extract bounardy and grad maps
bound_lh, _ = extract_parcellation(dataset, 'lh', run)
bound_rh, _ = extract_parcellation(dataset, 'rh', run)

# Combine surface meshes
coords_c, faces_c, scalar_c = combine_surfaces(coords_lh, faces_lh, bound_lh,
                                              coords_rh, faces_rh, bound_rh)

visualize_brain_surface(coords_c, faces_c, scalar_c, title='Boundary map ', cmap='cold_hot')



In [ ]:
from visualization import combine_surfaces
import numpy as np

run = "1"
dataset = "PPSFACE_N18"
inf_lh_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\lh.inflated"
inf_rh_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\\rh.inflated"
# 1. Read coordinates and faces as you do now
coords_lh, faces_lh = nib.freesurfer.read_geometry(inf_lh_path)
coords_rh, faces_rh = nib.freesurfer.read_geometry(inf_rh_path)
# Extract bounardy and grad maps
bound_lh, _ = extract_parcellation(dataset, 'lh', run)
bound_rh, _ = extract_parcellation(dataset, 'rh', run)

# 2. Compute the bounding box (or max extent) for the left hemisphere
#    so we know how far to shift the right hemisphere.
lh_max_x = np.max(coords_lh[:, 0])
lh_min_x = np.min(coords_lh[:, 0])
lh_width = lh_max_x - lh_min_x

# 3. Shift the right hemisphere by at least that width + a small margin
coords_rh[:, 0] += lh_width + 4  # 5 mm margin, for instance

# Now the two hemispheres won't overlap
coords_c, faces_c, scalar_c = combine_surfaces(coords_lh, faces_lh, bound_lh,
                                               coords_rh, faces_rh, bound_rh)
visualize_brain_surface(coords_c, faces_c, scalar_c, title='Boundary map', cmap='cold_hot')


## Dice Coef comparison

In [ ]:
# Dice coefficient Run1 vs Run2
from group_analysis import dice_coefficient
dataset = "PPSFACE_N20"

hemi = "lh"
run = "1"

surface_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\{hemi}.white"
inf_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\{hemi}.inflated"
# surface_path = r"D:\Data_Conn_Preproc\fsaverage6\surf\rh.inflated"
coords, faces = nib.freesurfer.read_geometry(surface_path)
coords_, faces_ = nib.freesurfer.read_geometry(inf_path)
graph = build_mesh_graph(faces)


bound = np.load(f"D:\Data_Conn_Preproc\{dataset}\group_parcellation\\boundary_map_group_run{run}_{hemi}.npy")
bound_smoothed = smooth_surface_graph(graph, bound, iterations=10)
label_bound_run1 = watershed_by_flooding(graph, bound_smoothed)


run = "2"
bound = np.load(f"D:\Data_Conn_Preproc\{dataset}\group_parcellation\\boundary_map_group_run{run}_{hemi}.npy")
bound_smoothed = smooth_surface_graph(graph, bound, iterations=10)
label_bound_run2 = watershed_by_flooding(graph, bound_smoothed)

dice_p = dice_coefficient((label_bound_run1>=0)*1, (label_bound_run2>=0)*1)
# dice_b = dice_coefficient((label_bound_run1<0)*1, (label_bound_run2<0)*1)

print(f"Number of parcels run1: {(np.unique(label_bound_run1)>=0).sum()}")
print(f"Number of parcels run2: {(np.unique(label_bound_run2)>=0).sum()}")


print(f"Run 1-2 Dice coefficient on parcels : {dice_p}")
# print(f"Run 1-2 Dice coefficient on boundaries: {dice_b}")


In [ ]:
# Dice coefficient PPSFACE_N18 vs PPSFACE_N20
dataset1 = "PPSFACE_N18"
dataset2 = "PPSFACE_N20"
"D:\Data_Conn_Preproc\PPSFACE_N18\group_parcellation\labels_group_run12_smooth10_lh.npy"
hemi = "lh"
parcel1 = np.load(f"D:\Data_Conn_Preproc\{dataset1}\group_parcellation\\labels_group_run12_smooth10_{hemi}.npy")
parcel2 = np.load(f"D:\Data_Conn_Preproc\{dataset2}\group_parcellation\\labels_group_run12_smooth10_{hemi}.npy")
dice_p = dice_coefficient((parcel1>=0)*1, (parcel2>=0)*1)

print(f"Number of parcels dataset1: {(np.unique(parcel1)>=0).sum()}")
print(f"Number of parcels dataset2: {(np.unique(parcel2)>=0).sum()}")
print(f"dataset 1-2 Dice coefficient on parcels : {dice_p}")

In [ ]:
# dice_rnd = []
dice_rnd_parc = []
for i in range(100):
    # Compare the Dice indice for random parcels of same size
    rnd_parc_1 = create_random_parcels(graph,n_clusters=(np.unique(label_bound_run1)>=0).sum())
    rnd_parc_2 = create_random_parcels(graph,n_clusters=(np.unique(label_bound_run2)>=0).sum())

    dice_p_ = dice_coefficient((rnd_parc_1>=0)*1, (rnd_parc_2>=0)*1)
    # dice_b_ = dice_coefficient((rnd_parc_1<0)*1, (rnd_parc_2<0)*1)
    dice_rnd_parc.append(dice_p_)
    # dice_rnd.append(dice_b_)
    
    # print(f"Random 1-2 Dice coefficient on parcels: {dice_b_}")
    # print(f"Random 1-2 Dice coefficient on parcels: {dice_p_}")
print(f"Random 1-2 Dice coefficient on parcels: mean {np.mean(dice_rnd_parc)} std {np.std(dice_rnd_parc)}")

In [ ]:
rh_pps18run1vsrun2 = 0.7728
rh_pps20run1vsrun2 = 0.7731
rh_pps18vspps20 = 0.7817

lh_pps18run1vsrun2 = 0.7716
lh_pps20run1vsrun2 = 0.7803
lh_pps18vspps20 = 0.7743


# plot the homogenity scores
x_null = np.ones(len(dice_rnd_parc)) + np.random.uniform(-0.05, 0.05, len(dice_rnd_parc))
x_model = np.array([1]) 

plt.figure(figsize=(2, 5))
offsets = [-0.02, 0, 0.02]

plt.scatter(x_model + offsets[0], [lh_pps18run1vsrun2], 
            color='c', marker='<', s=100, label='Run1 vs Run2 PPSFACE18')
plt.scatter(x_model + offsets[1], [lh_pps20run1vsrun2], 
            color='c', marker='>', s=100, label='Run1 vs Run2 PPSFACE20')
plt.scatter(x_model + offsets[2], [lh_pps18vspps20], 
            color='c', marker='s', s=100, label='PPSFACE20 vs PPSFACE18')

plt.scatter(x_null, dice_rnd_parc, color='black', alpha=0.7, label='Null Model, 100 trials')

# Customizing the plot
plt.xticks([1],'')  # Single x-tick since both are on the same column
plt.ylabel('Dice Coefficient')
plt.title('Consistency of Parcellations in Left Hemisphere')
plt.legend(
    loc='lower center',            
    bbox_to_anchor=(0.5, -0.15),   
    fancybox=True, shadow=True,    
    ncol=2                         
)
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Show the plot
plt.show()

## Create an homogenity metric like craddock

In [ ]:
# Load the parcellation for lh run1
dataset = "PPSFACE_N20"
hemi = "lh"
# run = "1"
surface_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\{hemi}.white"
inf_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\{hemi}.inflated"
# surface_path = r"D:\Data_Conn_Preproc\fsaverage6\surf\rh.inflated"
coords, faces = nib.freesurfer.read_geometry(surface_path)
coords_, faces_ = nib.freesurfer.read_geometry(inf_path)
graph = build_mesh_graph(faces)

# Extract parcellation and grad maps
label = np.load(f"D:\Data_Conn_Preproc\{dataset}\group_parcellation\\labels_group_run12_smooth10_{hemi}.npy")
# bound = np.load(f"D:\Data_Conn_Preproc\{ppsface}\group_parcellation\\boundary_map_group_run{run}_{hemi}.npy")
# bound_smoothed = smooth_surface_graph(graph, bound, iterations=10)
# label_bound = watershed_by_flooding(graph, bound_smoothed)

# Load all of the surf fmri data form run 1 lh
surf_fmri_list1 = extract_fmri_timeseries(dataset=dataset, hemisphere=hemi, run='1')
surf_fmri_list2 = extract_fmri_timeseries(dataset=dataset, hemisphere=hemi, run='2')
surf_fmri_list = surf_fmri_list1 + surf_fmri_list2

In [ ]:
print(dataset, '   ')
print('mean and var', surf_fmri_list[2].mean(), surf_fmri_list[2].var())
print('list length', len(surf_fmri_list))
print('number of parcels', (np.unique(label)>=0).sum())

homogeneity_model = homogeneity_craddock_rt(label, surf_fmri_list)
print(f"Model homogeneity: {homogeneity_model}")


In [ ]:
# Test the null model on the
homogenity_null_list = []
for n_sample in range(1, 1000):
    label_null = create_random_parcels(graph, n_clusters=(np.unique(label)>=0).sum()) # Create a null model
    homogeneity_null = homogeneity_craddock_rt(label_null, surf_fmri_list)
    homogenity_null_list.append(homogeneity_null)
    print(f"Null homogeneity: {homogeneity_null}")

np.save(f"D:\Data_Conn_Preproc\{dataset}\group_parcellation\{dataset}_homogeneity_{hemi}_1000.npy", 
        homogenity_null_list)


In [ ]:
homogenity_null_list = np.load(f"D:\Data_Conn_Preproc\{dataset}\group_parcellation\{dataset}_homogeneity_{hemi}_1000.npy")
print(np.mean(homogenity_null_list), np.std(homogenity_null_list))

In [ ]:
# plot the homogenity scores
x_null = np.ones(len(homogenity_null_list)) + np.random.uniform(-0.05, 0.05, len(homogenity_null_list))
x_model = np.array([1]) 

plt.figure(figsize=(3, 3))
plt.scatter(x_model, [homogeneity_model], color='red', s=100, label='My Model')
plt.scatter(x_null, homogenity_null_list, color='black', alpha=0.7, label='Null Model')

# Customizing the plot
plt.xticks([1],'')  # Single x-tick since both are on the same column
plt.ylabel('Homogeneity')
plt.title('Comparison of Homogeneity (PPSFACE20)')
plt.legend(
    loc='lower center',            
    bbox_to_anchor=(0.5, -0.15),   
    fancybox=True, shadow=True,    
    ncol=1                         
)
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Show the plot
plt.show()

# Is the subject level parcellation better for homogenity than group parcellation ?


In [ ]:
# Load the parcellation for lh run1
hemi = "lh"
run = "1"
surface_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\{hemi}.white"
inf_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\{hemi}.inflated"
# surface_path = r"D:\Data_Conn_Preproc\fsaverage6\surf\rh.inflated"
coords, faces = nib.freesurfer.read_geometry(surface_path)
coords_, faces_ = nib.freesurfer.read_geometry(inf_path)
graph = build_mesh_graph(faces)

# Extract bounardy and grad maps
bound, grad = extract_parcellation(hemi, run)

bound_smoothed = smooth_surface_graph(graph, bound, iterations=10)
grad_smoothed = smooth_surface_graph(graph, np.mean(grad,axis=1), iterations=10)

label_grad = watershed_by_flooding(graph, grad_smoothed)
label_bound = watershed_by_flooding(graph, bound_smoothed)

# Load all of the surf fmri data form run 1 lh
surf_fmri_list = extract_fmri_timeseries(hemi, run)

# subject 4 parcellation and data
bound_sub4 = np.load(r"D:\Data_Conn_Preproc\PPSFACE_N18\sub-04\sub-04_parcellation\boundary_map_sub04_run1_lh.npy")
surf_fmri_s4 = [surf_fmri_list[3]]
bound_smoothed_sub4 = smooth_surface_graph(graph, bound_sub4, iterations=10)
label_sub4 = watershed_by_flooding(graph, bound_smoothed_sub4)


bound_sub3 = np.load(r"D:\Data_Conn_Preproc\PPSFACE_N18\sub-03\sub-03_parcellation\boundary_map_sub03_run1_lh.npy")
surf_fmri_s3 = [surf_fmri_list[2]]
bound_smoothed_sub3 = smooth_surface_graph(graph, bound_sub3, iterations=10)
label_sub3 = watershed_by_flooding(graph, bound_smoothed_sub3)


bound_sub2 = np.load(r"D:\Data_Conn_Preproc\PPSFACE_N18\sub-02\sub-02_parcellation\boundary_map_sub02_run1_lh.npy")
surf_fmri_s2 = [surf_fmri_list[1]]
bound_smoothed_sub2 = smooth_surface_graph(graph, bound_sub2, iterations=10)
label_sub2 = watershed_by_flooding(graph, bound_smoothed_sub2)

In [ ]:
# compute the group parcellation homogenity on subject 4 fmri 
homogeneity_group4 = homogeneity_craddock_rt(label_bound, surf_fmri_s4)
homogeneity_n4 = homogeneity_craddock_rt(label_sub4, surf_fmri_s4)

print(f"Group homogeneity: {homogeneity_group4}")
print(f"Subject 4 homogeneity: {homogeneity_n4}") # IMPORTANT RESULTS HERE !!!!!!!!!!!!!!!!!

In [ ]:
# compute the group parcellation homogenity on subject 4 fmri 
homogeneity_group3 = homogeneity_craddock_rt(label_bound, surf_fmri_s3)
homogeneity_n3 = homogeneity_craddock_rt(label_sub3, surf_fmri_s3)

print(f"Group homogeneity: {homogeneity_group3}")
print(f"Subject 3 homogeneity: {homogeneity_n3}") # IMPORTANT RESULTS HERE !!!!!!!!!!!!!!!!!

In [ ]:
# compute the group parcellation homogenity on subject 4 fmri 
homogeneity_group2 = homogeneity_craddock_rt(label_bound, surf_fmri_s2)
homogeneity_n2 = homogeneity_craddock_rt(label_sub2, surf_fmri_s2)

print(f"Group homogeneity: {homogeneity_group2}")
print(f"Subject 3 homogeneity: {homogeneity_n2}") # IMPORTANT RESULTS HERE !!!!!!!!!!!!!!!!!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Sample data (replace these with actual values)
list_sublvl = [homogeneity_n2, homogeneity_n3, homogeneity_n4]
list_group = [homogeneity_group2, homogeneity_group3, homogeneity_group4]
labels = ["Parcellation sub-1", "Parcellation sub-2", "Parcellation sub-3"]

# Convert to numpy arrays
index = np.arange(len(labels))

# Plot
plt.figure(figsize=(8, 5))
plt.plot(index, list_sublvl, marker='o', linestyle='-', color='b', label="Subject-Level Parcellation")
plt.plot(index, list_group, marker='s', linestyle='-', color='r', label="Group-Level Parcellation")

# Labels and Title
plt.ylabel("Homogeneity Score")
plt.title("Comparison of Subject-Level vs Group-Level Parcellation")
plt.xticks(index, labels)
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)

# Show the plot
plt.show()


# Graph theory analysis

## 1 Metric Part
In this part we will anaylse the parcellation by amking it into a graph. 

For that, we average the time course in the parcels to have an average parcel time course

And then we make a correaltion of these time courses.


In [ ]:
import numpy as np
import networkx as nx
import nibabel as nib
from parcellation_surface.preprocessing_surface import extract_fmri_timeseries
from parcellation_surface.gradient import build_mesh_graph
from parcellation_surface.group_analysis import create_random_parcels

# Make it for PPSFACE18 on run 1 and run 2
ppsface = "PPSFACE_N20"

surface_path_lh = f"D:\Data_Conn_Preproc\\fsaverage6\surf\lh.white"
surface_path_rh = f"D:\Data_Conn_Preproc\\fsaverage6\surf\\rh.white"
_, faces_lh = nib.freesurfer.read_geometry(surface_path_lh)
_, faces_rh = nib.freesurfer.read_geometry(surface_path_rh)

# Load the parcellation for lh run1
label_lh = np.load(f"D:\Data_Conn_Preproc\{ppsface}\group_parcellation\labels_group_run12_smooth10_lh.npy")
label_rh = np.load(f"D:\Data_Conn_Preproc\{ppsface}\group_parcellation\labels_group_run12_smooth10_rh.npy")
label_rh[label_rh>=0] += np.max(label_lh) + 1 # 40k x 1, new values for the labels
# Load all of the surf fmri data form run 1 lh
surf_fmri_list_1_lh = extract_fmri_timeseries(dataset = ppsface, hemisphere = 'lh', run = '1')
surf_fmri_list_2_lh = extract_fmri_timeseries(dataset = ppsface, hemisphere = 'lh', run = '2')
surf_fmri_list_1_rh = extract_fmri_timeseries(dataset = ppsface, hemisphere = 'rh', run = '1')
surf_fmri_list_2_rh = extract_fmri_timeseries(dataset = ppsface, hemisphere = 'rh', run = '2')

# With a random parcellation
graph_lh = build_mesh_graph(faces_lh)
graph_rh = build_mesh_graph(faces_rh)
label_lh_rnd = create_random_parcels(graph_lh, (np.unique(label_lh)>=0).sum())
label_rh_rnd = create_random_parcels(graph_rh, (np.unique(label_rh)>=0).sum())
label_rh_rnd[label_rh_rnd>=0] += np.max(label_lh_rnd) + 1 # 40k x 1, new values for the labels (negatives val dont move)


label_rh = label_rh_rnd
label_lh = label_lh_rnd

In [ ]:
# PUT ALL TOGETHER
from parcellation_surface.group_analysis import parcel_correlation
from networkx.algorithms.community import greedy_modularity_communities
from networkx.algorithms.community.quality import modularity

# Metrix used for the graph theory part
clustering_coef_list = []
modularity_list = []
average_shortest_path_list = []
small_world_list = [] # sigma values


for subject in range(len(surf_fmri_list_1_lh)):
    print(f"Subject {subject}")
    # Get the parcellation correlatino for lh and rh
    # Group de run 1 and 2
    run1_lh = surf_fmri_list_1_lh[subject]
    run2_lh = surf_fmri_list_2_lh[subject]
    run12_lh = np.concatenate((run1_lh, run2_lh), axis=1) # 40k x 760
    run1_rh = surf_fmri_list_1_rh[subject]
    run2_rh = surf_fmri_list_2_rh[subject]
    run12_rh = np.concatenate((run1_rh, run2_rh), axis=1) # 40k x 760
    
    label_both_hemi = np.concatenate((label_lh, label_rh), axis=0) # 80k x 1
    
    run12 = np.concatenate((run12_lh, run12_rh), axis=0) # 80k x 760
    
    
    # Create the correlation matrix
    parcel_corr = parcel_correlation(label_both_hemi, run12)
    
    # Get the 15% percentile of most correlate values
    min_corr = np.percentile(parcel_corr, 80)
    parcel_corr[parcel_corr<min_corr] = 0
    # 
    distance = (parcel_corr>0)*1
    link_strength = distance
    # distance = 1-parcel_corr
    # link_strength = parcel_corr + 1
    assert np.allclose(distance, distance.T), "correlation matrix is not symmetric"

    # Create a grpah for clustering coef  shortest path length
    G = nx.from_numpy_array(link_strength)
    G.remove_edges_from(nx.selfloop_edges(G)) # remove self loops (diagonal)
    # Create a graph for shortest path length
    G_spl = nx.from_numpy_array(distance)
    G_spl.remove_edges_from(nx.selfloop_edges(G_spl)) # remove self loops (diagonal)
    # create a random unidirected graph with 15% density
    G = nx.erdos_renyi_graph((np.unique(label_both_hemi)>=0).sum(), 0.20) # used for comparison
    G_spl = nx.erdos_renyi_graph((np.unique(label_both_hemi)>=0).sum(), 0.20) # used for comparison
    
    
    # The clustering coef
    avg_clustering = nx.average_clustering(G, weight='weight')
    clustering_coef_list.append(avg_clustering)
    
    # The modularity
    communities = list(greedy_modularity_communities(G, weight='weight'))
    mod_value = modularity(G, communities, weight='weight')
    modularity_list.append(mod_value)
    
    # Avergae shortest path length
    if nx.is_connected(G_spl):
        avg_shortest_path = nx.average_shortest_path_length(G_spl, weight='weight')
    else:
        print('Not connected')
        components = list(nx.connected_components(G_spl))
        avg_lengths = [nx.average_shortest_path_length(G_spl.subgraph(comp), weight='weight')
                    for comp in components if len(comp) > 1]
        avg_shortest_path = np.mean(avg_lengths) if avg_lengths else None
    average_shortest_path_list.append(avg_shortest_path)
    
    # (Optional) Small-World Analysis.
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    G_rand = nx.gnm_random_graph(num_nodes, num_edges)
    rand_clustering = nx.average_clustering(G_rand)
    if nx.is_connected(G_rand):
        rand_shortest_path = nx.average_shortest_path_length(G_rand)
    else:
        comp_rand = list(nx.connected_components(G_rand))
        rand_lengths = [nx.average_shortest_path_length(G_rand.subgraph(comp))
                        for comp in comp_rand if len(comp) > 1]
        rand_shortest_path = np.mean(rand_lengths) if rand_lengths else None

    if rand_shortest_path is not None and avg_shortest_path is not None:
        sigma = (avg_clustering / rand_clustering) / (avg_shortest_path / rand_shortest_path)
    else:
        sigma = None
    small_world_list.append(sigma)

In [ ]:
# print the mean and std of each list 
print('for rnd graph on ppsface20, no weight 20percent connexions')
print(f"Mean clustering coef: {np.mean(clustering_coef_list)}", f"Std clustering coef: {np.std(clustering_coef_list)}")
print(f"Mean modularity: {np.mean(modularity_list)}", f"Std modularity: {np.std(modularity_list)}")
print(f"Mean average shortest path: {np.mean(average_shortest_path_list)}", f"Std average shortest path: {np.std(average_shortest_path_list)}")
print(f"Mean small world: {np.mean(small_world_list)}", f"Std small world: {np.std(small_world_list)}")

With 1-coor for distance, 1+corr for edge weight ---> keeping all of the links ---> bad idea
The distance is not valuable like that... the less percent of correlation we keep the best is the small worldness.

With all of the links : 

- Mean clustering coef: 0.5362756242701668 Std clustering coef: 0.014066995289751213
- Mean modularity: 0.04574750578995522 Std modularity: 0.0123708154468624
- Mean average shortest path: 0.9260425933152088 Std average shortest path: 0.02371690189496446
- Mean small world: 0.5798353331931781 Std small world: 0.02963548039694061


Random parcellation :

- Mean clustering coef: 0.268500971073205 Std clustering coef: 0.030815816609678116
- Mean modularity: 0.3858715669299432 Std modularity: 0.03755719653882077
- Mean average shortest path: 0.9075367719586135 Std average shortest path: 0.01660680582718532
- Mean small world: 3.712423345046655 Std small world: 0.487593423875435


for random parels on ppsface20
- Mean clustering coef: 0.27547801169118535 Std clustering coef: 0.03228586551964928
- Mean modularity: 0.38871774970003786 Std modularity: 0.039183588602628994
- Mean average shortest path: 0.9101651841808298 Std average shortest path: 0.014618571364787772
- Mean small world: 3.7895583358363147 Std small world: 0.5110324977906517

for  parels on ppsface20
- Mean clustering coef: 0.26451002929199907 Std clustering coef: 0.0342991075758053
- Mean modularity: 0.3996314985892854 Std modularity: 0.04169672463991428
- Mean average shortest path: 0.9147347562112222 Std average shortest path: 0.013401760386203608
- Mean small world: 3.618138570579604 Std small world: 0.5168140393944595

for  parels rnd on ppsface18, no weight 20percent connexions 
- Mean clustering coef: 0.5534898545587127 Std clustering coef: 0.02973890598361682
- Mean modularity: 0.3153656387982766 Std modularity: 0.036957117704235806
- Mean average shortest path: 1.9831311740597803 Std average shortest path: 0.06565853374026975
- Mean small world: 2.5347296782173587 Std small world: 0.06626107458262778

for  parels rnd on ppsface20, on weight 20percent connexions
Mean clustering coef: 0.5631306961800053 Std clustering coef: 0.03133179680242779
Mean modularity: 0.32931525373905596 Std modularity: 0.02867038544013402
Mean average shortest path: 1.959159886152849 Std average shortest path: 0.09042579496997746
Mean small world: 2.618559236539185 Std small world: 0.2238256408488425

for rnd graph on ppsface20, no weight 20percent connexions
Mean clustering coef: 0.20056889237233466 Std clustering coef: 0.0019105323167833586
Mean modularity: 0.06754303793904864 Std modularity: 0.0020174648024495236
Mean average shortest path: 1.8002356419413383 Std average shortest path: 0.00157101733352845
Mean small world: 0.9989269033708027 Std small world: 0.00326644634047313

## 2 Laplacian Eigenmodes

From the correlation matrix we can derive the main eigenmodes, a bit like the Haak et al. paper.

The correaltion matrix is transformed into a Laplacian graph.
Negatives values are dealt like in the Haak et al. paper.


In [ ]:
def parcel_correlation(group_parcel, 
                       surf_fmri):
    """
    Group the fMRI data into parcels and compute the correlation between parcels.
        group_parcel (np.ndarray): An array where each element indicates the parcel assignment 
                                   of the corresponding vertex in the surface fMRI data.
        surf_fmri (np.ndarray): Single Subject fmri data, A 2D array where each row represents a vertex and each column 
                                represents a time point of the fMRI data.
    Returns:
        np.ndarray: A 2D array representing the correlation matrix between the parcels.
    """
    # Get unique parcel indices greater than 0 (non-zero parcels)
    parcel_idx = np.unique(group_parcel[group_parcel >= 0])
    n_parcels = len(parcel_idx)
    
    # Initialize an array to store mean fMRI data for each parcel
    parcel_data = np.zeros((n_parcels, surf_fmri.shape[1]))
    
    # For each parcel, calculate the mean fMRI time series across vertices in the parcel
    for i, idx in enumerate(parcel_idx):
        # Boolean mask for vertices in the current parcel
        mask = group_parcel == idx
        # Compute the mean fMRI data for this parcel and store it
        parcel_data[i] = np.mean(surf_fmri[mask], axis=0)
        
        # Check for NaN values in the computed parcel data
        if np.isnan(parcel_data[i]).any():
            print(f'Parcel {idx} has NaN values')
    
    # Compute and return the correlation matrix between parcels
    parcel_corr = np.corrcoef(parcel_data)
    return parcel_corr

In [ ]:
import numpy as np
import networkx as nx
# from group_analysis import parcel_correlation

# Make it for PPSFACE18 on run 1 and run 2
ppsface = "PPSFACE_N20"
# Load the parcellation for lh run1
label_lh = np.load(f"D:\Data_Conn_Preproc\{ppsface}\group_parcellation\labels_group_run12_smooth10_lh.npy")
label_rh = np.load(f"D:\Data_Conn_Preproc\{ppsface}\group_parcellation\labels_group_run12_smooth10_rh.npy")
label_rh[label_rh>=0] += np.max(label_lh) + 1 # 40k x 1, new values for the labels
# Load all of the surf fmri data form run 1 lh
surf_fmri_list_1_lh = extract_fmri_timeseries(dataset = ppsface, hemisphere = 'lh', run = '1')
surf_fmri_list_2_lh = extract_fmri_timeseries(dataset = ppsface, hemisphere = 'lh', run = '2')
surf_fmri_list_1_rh = extract_fmri_timeseries(dataset = ppsface, hemisphere = 'rh', run = '1')
surf_fmri_list_2_rh = extract_fmri_timeseries(dataset = ppsface, hemisphere = 'rh', run = '2')


# Get the parcellation correlatino for lh and rh
subject = 0
# Group de run 1 and 2
run1_lh = surf_fmri_list_1_lh[subject]
run2_lh = surf_fmri_list_2_lh[subject]
run12_lh = np.concatenate((run1_lh, run2_lh), axis=1) # 40k x 760
run1_rh = surf_fmri_list_1_rh[subject]
run2_rh = surf_fmri_list_2_rh[subject]
run12_rh = np.concatenate((run1_rh, run2_rh), axis=1) # 40k x 760
label_both_hemi = np.concatenate((label_lh, label_rh), axis=0) # 80k x 1
run12 = np.concatenate((run12_lh, run12_rh), axis=0) # 80k x 760
parcel_corr = parcel_correlation(label_both_hemi, run12)

# Get the 20% percentile of most correlate values
min_corr = np.percentile(parcel_corr, 85)
parcel_corr[parcel_corr<min_corr] = 0


# Get the eigenmodes with a graph laplacian
# W = parcel_corr + 1 # to have values between 0 and 2 (or keep only the strongest connections)
W = parcel_corr
D = np.diag(np.sum(W,0))
L = np.subtract(D,W)
# Compute the eigenvectors and eigenvalues
eigvals, eigvecs = np.linalg.eig(L)
idx = eigvals.argsort()
eigvals = eigvals[idx]
eigvecs = eigvecs[:,idx]


In [ ]:
# Get the eigenmaps with a PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
eigvecs_pca = pca.fit_transform(eigvecs)
eigvecs_pca.shape

In [ ]:

fielder_lh = np.zeros_like(label_lh, dtype=float)
fielder_lh.shape

In [ ]:
# the fielder vector is a 465 x 1 vector, i want to attribute the value of the parcels to the vertices

# Get the fielder vector
fielder = eigvecs[:,12] 

fielder_lh = np.zeros_like(label_lh, dtype=float)

label_values = np.unique(label_lh[label_lh>= 0])

for i in label_values:
    # fielder_lh[label_lh == i] = fielder[i]
    # fielder_lh[label_lh == i] = parcel_corr[0,i]
    fielder_lh[label_lh == i] = eigvecs_pca[i,1]
    

In [ ]:
hemi = "lh"
surface_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\{hemi}.white"
inf_path = f"D:\Data_Conn_Preproc\\fsaverage6\surf\{hemi}.inflated"
# surface_path = r"D:\Data_Conn_Preproc\fsaverage6\surf\rh.inflated"
coords_lh, faces_lh = nib.freesurfer.read_geometry(inf_path)
visualize_brain_surface(coords_lh, faces_lh, fielder_lh, title='Fielder Vector')